In [ ]:
import pandas as pd
import numpy as np
from numpy.polynomial import Polynomial                      # Current way to calculate polynomials for graph overlays - replaces polyfit

import streamlit as st
import matplotlib.pyplot as plt
from pathlib import Path

#can drop this if you want popups you can resize
%matplotlib inline 

In [ ]:
file_path = r'C:\Users\justi\Documents\Projects\Data\Baseball_databank'

teams_path = file_path + '\Teams.csv'
salary_path = file_path + '\Salaries.csv'
batting_path = file_path + '\Batting.csv'

In [ ]:
teams_df = pd.read_csv(teams_path)
teams_df = teams_df[teams_df['yearID'] >= 1985]
teams_df.info()

In [ ]:
salary_df = pd.read_csv(salary_path)
salary_df = salary_df[salary_df['yearID'] >= 1985]
salary_df.info()

In [ ]:
# How have salaries evolved by division over the years?
# Sum of salaries by division by year, then dig in by team

# .reset_index() converts back to a pandas dataframe, otherwise it would have been a series

team_salary_by_year_df = salary_df.groupby(['yearID','teamID','lgID'])['salary'].sum().reset_index()
team_salary_by_year_df.head()

In [ ]:
# cool reference material for graphs here:
# https://medium.com/dataexplorations/reproducing-the-baseball-salary-visualization-from-fivethirtyeight-in-matplotlib-1af449beb409
# Lets try and recreate!

team_salary_by_year_df['salary_zscore'] = team_salary_by_year_df.groupby(['yearID'])['salary'].transform(
    lambda x: (x - x.mean()) / x.std()
)

team_salary_by_year_df.head()

In [ ]:
teams_df.head()

In [ ]:
#now, lets get win rates. First step is grabbing season wins and losses from the teams table and cleaning it up
teams_df = teams_df[['yearID','lgID','teamID','G','W','L']]
teams_df = teams_df[(teams_df['yearID'] >= 1985)].reset_index().drop('index',axis=1)
teams_df.head()

In [ ]:
# Now that we have wins and total # games (W and G)
teams_df['win_rate'] = teams_df['W']/teams_df['G']
teams_df.head()

In [ ]:
merged_df = team_salary_by_year_df.merge(teams_df, on = ['yearID','lgID', 'teamID']).reset_index().drop('index',axis = 1)
merged_df.head()

In [ ]:
# Lets start with one scatterplot for the Phillies (go Matt). THink it's PHI but lets check
a = merged_df['teamID'].unique()
print(sorted(a))

In [ ]:
phillies = merged_df[(merged_df['teamID'] == 'PHI')]

In [ ]:
# Now, lets graph this!
plt.scatter(x = phillies['salary_zscore'], y = phillies['win_rate'], alpha = 0.5)
plt.title("PHI")
plt.xlabel("Standardized Salaries")
plt.ylabel("Win Rate")

#plotting a grid axes (win rate at y=0.5, Salary zscore at x=0)
# this also looks like it forces the graph space to be square and centered somewhat - cool
plt.hlines(0.5,-2,2)
plt.vlines(0,0.4,0.6)

# I want to add a line of best fit to the graph for the win rate. Going to use the polyfit function in NumPy, although this is outdated. Come back and update
z = np.polyfit(phillies['salary_zscore'],phillies['win_rate'], 1)
p = np.poly1d(z)

plt.plot(phillies['salary_zscore'], p(phillies['salary_zscore']),alpha=0.8,c ='#1f77b4',linewidth =3)

plt.show()

In [ ]:
# Lets do this again, but tweaking a bit to remove the border and add a gridline.
# Also, switching from deprecated polyfit method to NumPy's polynomial.Polynomial method
# Finally, remove the border on the graph using teh ax.spines method

# not sure how declaring fig, ax and using subplots allowes the use of spines 
fig, ax = plt.subplots(figsize=(5, 5))
ax.scatter(x=phillies['salary_zscore'], y=phillies['win_rate'], alpha=0.5)
plt.title('PHI')
plt.ylabel('Win Rate')
plt.xlabel('Standardized Salaries')

plt.hlines(0.5, -2, 2)
plt.vlines(0,0.3,0.7)

# To create my polynomial overlay and set it's visual appearance
degree = 2
polynomial = np.polynomial.Polynomial.fit(phillies['salary_zscore'], phillies['win_rate'], degree)
ax.grid(color='grey', linestyle ='-', linewidth=0.25, alpha=0.5)

# Remove all spines at once from the graph
for spine in ax.spines.values():
    spine.set_visible(False)

plt.plot(phillies['salary_zscore'], polynomial(phillies['salary_zscore']),alpha=0.8,c ='#1f77b4',linewidth =3)

plt.show()

In [ ]:
# Print the polynomial and equation
print(f"Polynomial: {polynomial}")
print(f"Coefficients: {polynomial.coef}")

# Just for fun, what's the R-Squared for this estimate?
y = phillies['salary_zscore']
y_pred = polynomial(y)
ss_res = np.sum((y - y_pred) ** 2)
ss_tot = np.sum((y - np.mean(y)) ** 2)
r_squared = 1 - (ss_res / ss_tot)
print(f"R-Squared: {r_squared}")

# Odd NumPy doesn't have an R-squared method built in. Sklearn.metrics r2_score

In [ ]:
# Now, lets make a function that lets us do this for any team we want

def plot_team_row(team_df, ax, degree, team_color = '#1f77b4'):
    '''
        This creates a subplot for 1 team.
        team_df is the dataframe with info for that team
        ax lets us specify the subplot location
        degree is mine, lets us specify the degree of the polynomial we want (probably always going to do 2)
        team_color is for, well, the team color
    '''
    #generate line of best fit & plot
    z = np.polynomial.Polynomial.fit(team_df['salary_zscore'],team_df['win_rate'], degree)
    ax.plot(team_df['salary_zscore'],polynomial(team_df['salary_zscore']), alpha = 0.8, c=team_color, linewidth = 3)
    